# Number of Matching Subsequences

Given a string s and an array of strings words, return the number of words[i] that is a subsequence of s.

A subsequence of a string is a new string generated from the original string with some characters (can be none) deleted without changing the relative order of the remaining characters.

For example, "ace" is a subsequence of "abcde".
 

Example 1:

Input: s = "abcde", words = ["a","bb","acd","ace"]
Output: 3
Explanation: There are three strings in words that are a subsequence of s: "a", "acd", "ace".
Example 2:

Input: s = "dsahjpjauf", words = ["ahjpjau","ja","ahbwzgqnuk","tnmlanowax"]
Output: 2
 

Constraints:

1 <= s.length <= 5 * 104
1 <= words.length <= 5000
1 <= words[i].length <= 50
s and words[i] consist of only lowercase English letters.

In [ ]:
def numMatchingSubseq(self, s: str, words: List[str]) -> int:
        count=0
        for i in range(len(words)):
            m=s
            k=""
            for j in words[i]:
                if j in m:
                    m=m[m.index(j)+1:]
                    k+=j
            if k==words[i]:
                count+=1
        return (count)

In [ ]:
class Solution:
    def numMatchingSubseq(self, s: str, words: List[str]) -> int:
        ctr = 0
        mp = defaultdict(list)
        
        #STEP1 Create a map to store the indexes of characters in string
        for i,c in enumerate(s):
            mp[c].append(i)
            
        #STEP2 Iterate each word in words[] to check if it is subsequence
        for w in words:
            pre_pos = -1    #To keep track of the position of matched character
            yes = True
            for c in w:
                if c not in mp:
                    yes = False
                    break
                index = bisect.bisect_right(mp[c], pre_pos) #Perform binary search of previous position on list of indices of current character
                
                if index == len(mp[c]):     #Binary search failed
                    yes = False
                    break
                pre_pos = mp[c][index]      #Update previous position of matched character
            if yes:
                ctr += 1                    
        return ctr

# Time Complexity Analysis:
We assume the length of string s is L, the length of word list words is M and the maximum length of word is N.

Binary Search:
In this case, we do the binary search for each character in each word in words list. Then, for each binary search, we assume the length of the array we applying the binary search is the whole string s (where it can't be in most cases). So, the totaly time is MN logL, which is feasible by the given lengths.

Array Method:
This time, we want to iterate each character in each word and also walk through the entire string s. So, the total time should be L + MN, which is slight faster than the first case.

In [ ]:
class Solution:
    def numMatchingSubseq(self, s: str, words: List[str]) -> int:
        # Solution 1: Binary Search
        # 620ms + 17.3MB
        ind = defaultdict(list)
        for i, char in enumerate(s):
            ind[char].append(i)
        res = 0
        def check(word):
            s = 0
            for char in word:
                if not ind[char]:
                    return False
                pos = bisect_left(ind[char], s)
                if pos==len(ind[char]):
                    return False
                s = ind[char][pos]+1
            return True
        for word in words:
            if check(word):
                res += 1
        return res
        
        # Solution 2: Tricky Array Method
        # 508ms + 15.7MB
        title = defaultdict(list)
        for word in words:
            title[word[0]].append(word)
        res = 0
        for char in s:
            word_list = title[char]
            title[char] = []
            for word in word_list:
                if len(word)==1:
                    res += 1
                else:
                    title[word[1]].append(word[1:])
        return res

Algorithm:

We first count the occurences of a word in words array and store in a final_count dictionary.
Then we iterate throught the set(words).
In the for loop we use iter which creates an iterator and will give us whether a letter of a word is present or not.
If it is present we increment the value of i by the number of its occurences that we have stored in the final_count dictionary.


Time: O(N*M)
Space: O(N*M)
where N - length of the words array, M - length of the string

In [ ]:
class Solution:
    def numMatchingSubseq(self, s: str, words: List[str]) -> int:
        
        i = 0
        
        final_count = Counter(words)
        
        for word in set(words):
            it = iter(s)
            if all(letter in it for letter in word):
                i += final_count[word]
        
        return i

In [1]:
5//2


2

In [2]:
4//2

2

# Flip String to Monotone Increasing


A string of '0's and '1's is monotone increasing if it consists of some number of '0's (possibly 0), followed by some number of '1's (also possibly 0.)

We are given a string S of '0's and '1's, and we may flip any '0' to a '1' or a '1' to a '0'.

Return the minimum number of flips to make S monotone increasing.

 

Example 1:

Input: "00110"
Output: 1
Explanation: We flip the last digit to get 00111.
Example 2:

Input: "010110"
Output: 2
Explanation: We flip to get 011111, or alternatively 000111.
Example 3:

Input: "00011000"
Output: 2
Explanation: We flip to get 00000000.
 

Note:

1 <= S.length <= 20000
S only consists of '0' and '1' characters.

## Approach 1: Prefix Sums
Intuition

For say a 5 digit string, the answer is either '00000', '00001', '00011', '00111', '01111', or '11111'. Let's try to calculate the cost of switching to that answer. The answer has two halves, a left (zero) half, and a right (one) half.

Evidently, it comes down to a question of knowing, for each candidate half: how many ones are in the left half, and how many zeros are in the right half.

We can use prefix sums. Say P[i+1] = A[0] + A[1] + ... + A[i], where A[i] = 1 if S[i] == '1', else A[i] = 0. We can calculate P in linear time.

Then if we want x zeros followed by N-x ones, there are P[x] ones in the start that must be flipped, plus (N-x) - (P[N] - P[x]) zeros that must be flipped. The last calculation comes from the fact that there are P[N] - P[x] ones in the later segment of length N-x, but we want the number of zeros.

## Algorithm

For example, with S = "010110": we have P = [0, 0, 1, 1, 2, 3, 3]. Now say we want to evaluate having x=3 zeros.

There are P[3] = 1 ones in the first 3 characters, and P[6] - P[3] = 2 ones in the later N-x = 3 characters.

So, there is (N-x) - (P[N] - P[x]) = 1 zero in the later N-x characters.

We take the minimum among all candidate answers to arrive at the final answer.


## Complexity Analysis

Time Complexity: O(N)O(N), where NN is the length of S.

Space Complexity: O(N)O(N).

In [ ]:
# didnot works
def minFlipsMonoIncr(self, S: str) -> int:
    num_ones_seen = 0
    min_cost = 0
    for letter in S:
        if letter == '1':
            num_ones_seen += 1
        else:
            min_cost = min(min_cost + 1, num_ones_seen)
    return min_cost

In [ ]:
class Solution:
    def minFlipsMonoIncr(self, S: str) -> int:
        nums = list(map(int, list(S)))
        n = len(S)
        dp = [[0 for _ in range(n)] for _ in range(2)]
        dp[0][0] = int(nums[0] != 0)
        dp[1][0] = 1 - dp[0][0]
        for i in range(1,n):
            if nums[i]==1:
                dp[0][i] = dp[0][i-1] + 1
                dp[1][i] = min(dp[1][i-1], dp[0][i-1])
            else:
                dp[0][i] = dp[0][i-1]
                dp[1][i] = min(dp[1][i-1], dp[0][i-1]) + 1
        return min(dp[0][n-1], dp[1][n-1])

In [ ]:
class Solution:
    def minFlipsMonoIncr(self, S: str) -> int:
        zeroes = S.count('0')
        min_flips, prefix_ones = float(inf), 0
        
        for i,num in enumerate(S):
            suffix_zeroes = zeroes - i + prefix_ones
            min_flips = min(prefix_ones + suffix_zeroes - (not int(num)), min_flips)            
            prefix_ones += int(num)

        return min_flips

In [ ]:
class Solution:
def minFlipsMonoIncr(self, S: str) -> int:
    left=0
    right=S.count('1')
    ls=[]
    ls.append(len(S)-right)
    
    for i in range(len(S)):
        if S[i]=='0':
            left+=1
        else:
            right-=1
        
        ls.append((i+1-left)+(len(S)-i-1-right))
            
    return min(ls)

### *We tend to find an index that all the "1" on the left will be flipped to "0" and all the "0" on the right will be flipped to "1". Keep in mind that this index may not exit, this is equivalent to say flip all the "0" to "1" or all the "1" to "0".
*Below "left" stores the number of "0" and "1" on the left(including current position), while "right" stores the number of "0" and "1" on the right(do not include current position). The number of flips we have to take is left[1]+right[0].

In [ ]:
def minFlipsMonoIncr(self, S: str) -> int:
    left = [0,0]
    right = [S.count("0"),S.count("1")]
    diff = min(right[0],right[1])
    for x in S:
        if x == "0":
            left[0]+=1
            right[0]-=1
        else:
            left[1]+=1
            right[1]-=1
        diff = min(diff, left[1]+right[0])
    return diff

# Partition Array Into Three Parts With Equal Sum

Given an array of integers arr, return true if we can partition the array into three non-empty parts with equal sums.

Formally, we can partition the array if we can find indexes i + 1 < j with (arr[0] + arr[1] + ... + arr[i] == arr[i + 1] + arr[i + 2] + ... + arr[j - 1] == arr[j] + arr[j + 1] + ... + arr[arr.length - 1])

 

Example 1:

Input: arr = [0,2,1,-6,6,-7,9,1,2,0,1]
Output: true
Explanation: 0 + 2 + 1 = -6 + 6 - 7 + 9 + 1 = 2 + 0 + 1
Example 2:

Input: arr = [0,2,1,-6,6,7,9,-1,2,0,1]
Output: false
Example 3:

Input: arr = [3,3,6,5,-2,2,5,1,-9,4]
Output: true
Explanation: 3 + 3 = 6 = 5 - 2 + 2 + 5 + 1 - 9 + 4
 

Constraints:

3 <= arr.length <= 5 * 104
-104 <= arr[i] <= 104

In [ ]:
class Solution:
    def canThreePartsEqualSum(self, arr: List[int]) -> bool:
        s = sum(arr)
        if (s%3)!=0:
            return False
        avg = s/3
        count = 0
        curr = 0
        for i in arr:
            curr +=i
            if curr==avg:
                count+=1
                curr=0
        if count>=3:
            return True

In [ ]:
def canThreePartsEqualSum(self, arr: List[int]) -> bool:
	return(s:=[sum(arr)])[0]%3==0and len([s.append(1)for r in accumulate(arr)if r/len(s)==s[0]/3])>2

In [ ]:
class Solution:
    def canThreePartsEqualSum(self, arr: List[int]) -> bool:
        sm=0
        for x in arr:
            sm+=x
        sm/=3
        sub=0
        aux=0
        for i in range(0,len(arr)):
            if sub<2 :
                aux+=arr[i]
                if aux==sm:
                    aux=0
                    sub+=1
                    if sub==2 and i==len(arr)-1:
                        return False
            else :
                aux+=arr[i]
                
        return aux==sm   and sub==2

In [ ]:
s = sum(arr)
if s % 3 != 0:
	return False
average = s / 3
cur = 0
count = 0
for num in arr:
	cur += num
	if cur == average:
		cur = 0
		count += 1
return count >= 3

In [ ]:
class Solution:
    def canThreePartsEqualSum(self, arr: List[int]) -> bool:
        total = sum(arr)
        if total % 3 != 0 : 
            return False
        
        desiredSubArrayTotal, subTotal, partition = total // 3, 0, 0        
        
        for num in arr : 
            subTotal += num 
            if subTotal == desiredSubArrayTotal : 
                partition += 1
                subTotal = 0
                
        
        return True if subTotal == 0 and partition >= 3 else False

# last question acccidently saved in Day 2